In [9]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import csv
import pandas as pd
from datetime import datetime, date

In [42]:
class Stock_CAC40:
    
    def __init__(self):
        self.stock_list = ["AC.PA", "AI.PA", "AIR.PA", "ATO.PA", "CS.PA", "BNP.PA", "EN.PA", "CAP.PA", "CA.PA", "ACA.PA", 
          "BN.PA", "ENGI.PA", "EI.PA", "KER.PA", "OR.PA", "LHN.PA", "LR.PA", "MC.PA", "ML.PA", "ORA.PA",
          "RI.PA", "UG.PA", "PUB.PA", "RNO.PA", "SAF.PA", "MT.AS", "UL.AS", "SOLB.BR", "SGO.PA", "SAN.PA",
          "SU.PA", "GLE.PA", "SW.PA", "STM.PA", "FTI.PA", "FP.PA", "FR.PA", "VIE.PA", "DG.PA", "VIV.PA"]
        self.browser = None
        
    def scroll_page(self):
        #Nous allons scroller la page jusqu'à l'arret
        lenOfPage = self.browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);var lenOfPage=document.documentElement.scrollHeight;return lenOfPage;")
        match=False    #match indique si on est arrivé au bout de la page ou non
        while(match==False):
            lastCount = lenOfPage
            time.sleep(0.1)
            #Code en entrée en JavaScript
            lenOfPage = self.browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);var lenOfPage=document.documentElement.scrollHeight;return lenOfPage;")
            if lastCount==lenOfPage:
                match=True
    
    def update_list_stock_CAC40(self):
        return "update not operational"
        self.browser = webdriver.Chrome('C:/Users/Julien Veron Vialard/Documents/ProjetS8 - Github/projets8/chromedriver.exe')
        self.browser.get("https://finance.yahoo.com/quote/%5EFCHI/components")
        temp = True
        while temp == True:
            try:
                element = self.browser.find_element_by_class_name('btn-primary')
                element.click()
                time.sleep(1)
            except:
                temp = False
                pass
        self.scroll_page()
        soup = BeautifulSoup(self.browser.page_source, 'html.parser')
        
        print(soup)

        tableau = list(soup.find_all('tbody'))
        tableau = tableau[0]
        tab_content = list(tableau.children)
        n = len(tab_content)
        lines_list = list()
        for i in range(n):
            lines_list.append(list(tab_content[i].children))

        Stocks = list()
        for i in range(len(lines_list)):
            temp = []
            for j in range(len(lines_list[i])):
                temp.append(lines_list[i][j].get_text())
            Stocks.append(temp[0])
            
        self.stock_list = Stocks

In [12]:
class Updater_stock:
    
    def __init__(self):
        self.browser = webdriver.Chrome('C:/Users/Julien Veron Vialard/Documents/ProjetS8 - Github/projets8/chromedriver.exe')
        self.soup = None
        self.historical = list()
        self.header = ["Date", "Open", "High", "Low", "Close", "Adj close", "Volume"]
        self.stock_name = None        
        
    def update_stock(self, stock_name):
        self.stock_name = stock_name
        self.access_to_page()
        time.sleep(1)
        
        self.scroll_page()
        self.get_soup()
        self.get_historical()
        self.processing_historical()
        self.export_to_csv()
        # réinitialiser paramètres
        self.soup = None
        self.historical = list()
        self.stock_name = None
        
    def access_to_page(self):
        self.browser.get("https://finance.yahoo.com/quote/" + self.stock_name + "/history?period1=1364338800&period2=1522101600&interval=1d&filter=history&frequency=1d")
        temp = True
        while temp == True:
            try:
                element = self.browser.find_element_by_class_name('btn-primary')
                element.click()
                time.sleep(1)
            except:
                temp = False
                pass
        return
        
    def scroll_page(self):
        #Nous allons scroller la page jusqu'à l'arret
        lenOfPage = self.browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);var lenOfPage=document.documentElement.scrollHeight;return lenOfPage;")
        match=False    #match indique si on est arrivé au bout de la page ou non
        while(match==False):
            lastCount = lenOfPage
            time.sleep(0.1)
            #Code en entrée en JavaScript
            lenOfPage = self.browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);var lenOfPage=document.documentElement.scrollHeight;return lenOfPage;")
            if lastCount==lenOfPage:
                match=True
    
    def get_soup(self):
        self.soup = BeautifulSoup(self.browser.page_source, 'html.parser')
        
    def get_historical(self):
        #On récupère le tableau
        tableau = list(self.soup.find_all('tbody'))
        tableau = tableau[1]
        tab_content = list(tableau.children)

        #On récupère les lignes du tableau 
        n = len(tab_content)
        lines_list = list()
        for i in range(n):
            lines_list.append(list(tab_content[i].children))

        #On crée le tableau des données historiques
        for i in range(len(lines_list)):
            temp = []
            for j in range(len(lines_list[i])):
                temp.append(lines_list[i][j].get_text())
            self.historical.append(temp)

    def processing_historical(self):
        """Traitement du contenu du tableau"""
        n = len(self.historical)
        i = 0
        compteur = 0
        while compteur < n:
            temp = self.historical[i]
            if len(temp)>2:
                temp[0] = temp[0].replace(',','')
                temp[0] = datetime.strptime(temp[0] , "%b %d %Y").date()
                c = temp[-1]
                if c == '-':
                    self.historical.remove(temp)
                    compteur +=1
                    continue
                else:
                    c = int(c.replace(',',''))
                    temp[-1] = c
                for j in range(1,6):
                    if temp[j] == '-':
                        self.historical.remove(temp)
                        i-=1
                        break
                    else:
                        temp[j] = float(temp[j])
                i+=1
            else:
                self.historical.remove(temp)
            compteur +=1
    
    def export_to_csv(self):
        ### Transformation en DataFrame
        Frame = pd.DataFrame(data = self.historical, columns = self.header)

        ### Sortie d'un CSV
        Frame.to_csv(path_or_buf = "data/" + self.stock_name + ".csv", header = True, index = False)

In [14]:
class Updater_historical_data:
    
    def __init__(self):
        self.updater_stock = Updater_stock()      
        
    def update(self, stock_names):  # stock_names = liste des noms des stocks que l'on souhaite updater
        for stock_name in stock_names:
            self.updater_stock.update_stock(stock_name)